## 폐암 수술환자 생존 예측
- 데이터 준비

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import tensorflow as tf

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

#그래프 패키지 
import matplotlib.pyplot as plt 
%matplotlib inline 
# 한글 문제 해결
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')# 한글 문제 해결
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')


# 동시 출력
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

1. 딥러닝 모델 설정
2. 모델 컴파일
3. 모델 실행 (학습)

In [3]:
# 실행할 때마다 같은 결과를 출력하기 위해 설정
tf.random.set_seed(3)

In [4]:
# 준비된 수술 환자 데이터를 불러 오기
Data_set = np.loadtxt("./data/ThoraricSurgery.csv", delimiter=",")
type(Data_set)
len(Data_set)


numpy.ndarray

470

In [5]:
Data_set

array([[293.  ,   1.  ,   3.8 , ...,   0.  ,  62.  ,   0.  ],
       [  1.  ,   2.  ,   2.88, ...,   0.  ,  60.  ,   0.  ],
       [  8.  ,   2.  ,   3.19, ...,   0.  ,  66.  ,   1.  ],
       ...,
       [406.  ,   6.  ,   5.36, ...,   0.  ,  62.  ,   0.  ],
       [ 25.  ,   8.  ,   4.32, ...,   0.  ,  58.  ,   1.  ],
       [447.  ,   8.  ,   5.2 , ...,   0.  ,  49.  ,   0.  ]])

In [6]:
# X와 Y로 구분하여 저장
X=Data_set[:,0:17]
Y=Data_set[:,17]

1. 딥러닝 모델 설정
- 레이어 구성 시작 객체 생성 (Sequential())
    - model = Sequential()
- Sequential model에 층을 추가
    - 마지막 층 (출력증)을 제외한 나머지 층은 은닉층으로 처리
    - Sequential() 객체의 add() 함수를 이용해서 추가
        - 특별한 알고리즘을 사용하는 경우가 아니라면 Dense() 이용
        - ex. model.add(Dense(출력 갯수, 첫층이라면 input_dim=컬럼수, activation='relu'))
        - model.add(Dense(30, input_dim=17, activation='relu'))
    - 추가하는 층이 출력층 (마지막 층)이라면 출력 갯수는 데이터에 따라 달라짐
        - 이진 분류인 경우는 1/ 다중분류라면 분류값 갯수 만큼 출력을 설정
        - model.add(Dense(1, activation='sigmoid'))

In [8]:
# (1) 모델 설정
model = Sequential()
model.add(Dense(30, input_dim=17, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # 0과 1로 분류일 경우 (다중 분류도 대부분은) sigmoid

2. 모델 compile
**Compile**  


- 앞에서 지정한 모델이 효과적으로 구현될 수 있도록
- 여러 가지 환경 설정하면서 실행가능한 상태가 되도록 만들어주는 단계
- 어떤 오차 함수를 사용할지 : loss:binary_crossentropy - log 형식을 사용하는 cost function
    - binary(두개의 결과)결과에 사용
    - 3이상이 결과에 대한 loss function은 categorical_crossentropy 사용
    - 다른 예측인경우 mean_squared_error 를 사용할 수 도 있음
- 어떤 최적화 방법을 사용할지 :optimizer:adam 
    - adam : 각 파라미터마다 다른 크기의 업데이트를 적용하는 방법이다.
- 수행 결과 평가는 어떤 방법을 사용할지 : metircs : accuracy   

를 결정


In [9]:
# (2) 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

3. 모델 실행 (학습)  
model.fit(X, Y, epochs=100, batch_size=10) 
fit() : 데이터 입력하면 앞에서 설정한 대로 실행되고 결과 출력



- epoch : 에포크
    - 학습 프로세스가 모든 샘플에 대해 한 번 실행하는 것
    - epoch= 100 : 각 샘플이 처음부터 끝까지 100번 재사용될 때까지 반복한다는 의미   
    
    
- batch_size
    - 샘플을 한 번에 몇 개씩 처리할 것인지 설정
    - batch_size = 10 : 전체 ~개의 샘플을 10개씩 끊어서 집어 넣으라는 의미
    - 배치 사이즈가 너무 크면 학습 속도가 느려지고, 
    - 너무 작으면 각 실행 값의 편차가 생겨서 전체 결과값이 불안정해질 수 있음
    - 사용하는 메모리가 감당할 만큼의 배치 사이즈로 설정


In [13]:
# (3) 모델 실행(학습)
hist = model.fit(X,Y, epochs=100, batch_size=10)
# 총 470 레코드 - batch _size가 10이므로 47개의 sample 그룹이 생성 10개가 47스텝으로 들어가짐 

Epoch 1/100
47/47 [==============================] - 0s 499us/step - loss: 0.3787 - accuracy: 0.8574
Epoch 2/100
47/47 [==============================] - 0s 434us/step - loss: 0.3975 - accuracy: 0.8532
Epoch 3/100
47/47 [==============================] - 0s 390us/step - loss: 0.3847 - accuracy: 0.8532
Epoch 4/100
47/47 [==============================] - 0s 406us/step - loss: 0.4204 - accuracy: 0.8468
Epoch 5/100
47/47 [==============================] - 0s 390us/step - loss: 0.3858 - accuracy: 0.8574
Epoch 6/100
47/47 [==============================] - 0s 369us/step - loss: 0.3732 - accuracy: 0.8532
Epoch 7/100
47/47 [==============================] - 0s 369us/step - loss: 0.3958 - accuracy: 0.8511
Epoch 8/100
47/47 [==============================] - 0s 390us/step - loss: 0.3841 - accuracy: 0.8574
Epoch 9/100
47/47 [==============================] - 0s 359us/step - loss: 0.3705 - accuracy: 0.8489
Epoch 10/100
47/47 [==============================] - 0s 390us/step - loss: 0.3835 - accura

47/47 [==============================] - 0s 390us/step - loss: 0.3754 - accuracy: 0.8660
Epoch 82/100
47/47 [==============================] - 0s 369us/step - loss: 0.3861 - accuracy: 0.8638
Epoch 83/100
47/47 [==============================] - 0s 390us/step - loss: 0.3675 - accuracy: 0.8660
Epoch 84/100
47/47 [==============================] - 0s 369us/step - loss: 0.4042 - accuracy: 0.8532
Epoch 85/100
47/47 [==============================] - 0s 369us/step - loss: 0.3788 - accuracy: 0.8532
Epoch 86/100
47/47 [==============================] - 0s 369us/step - loss: 0.3681 - accuracy: 0.8660
Epoch 87/100
47/47 [==============================] - 0s 369us/step - loss: 0.3737 - accuracy: 0.8511
Epoch 88/100
47/47 [==============================] - 0s 380us/step - loss: 0.3807 - accuracy: 0.8574
Epoch 89/100
47/47 [==============================] - 0s 390us/step - loss: 0.3680 - accuracy: 0.8617
Epoch 90/100
47/47 [==============================] - 0s 390us/step - loss: 0.3711 - accuracy: 

In [11]:
hist.history['loss'] 

[0.6482216715812683,
 0.48902347683906555,
 0.44164735078811646,
 0.4862949252128601,
 0.4430175721645355,
 0.43030914664268494,
 0.4421369135379791,
 0.43626195192337036,
 0.4164518117904663,
 0.4317093789577484,
 0.445814847946167,
 0.4384354054927826,
 0.46514859795570374,
 0.44745567440986633,
 0.493400901556015,
 0.4471505284309387,
 0.47471120953559875,
 0.44875475764274597,
 0.440741628408432,
 0.4379560053348541,
 0.43314385414123535,
 0.42423954606056213,
 0.4183190166950226,
 0.43033096194267273,
 0.42463505268096924,
 0.43353161215782166,
 0.4492495357990265,
 0.42414700984954834,
 0.42123445868492126,
 0.41357138752937317,
 0.4388585090637207,
 0.4187561869621277,
 0.4427948594093323,
 0.41033434867858887,
 0.417768657207489,
 0.41362571716308594,
 0.44290390610694885,
 0.42581018805503845,
 0.47140905261039734,
 0.40409055352211,
 0.4209672212600708,
 0.4418415427207947,
 0.40958476066589355,
 0.4032115936279297,
 0.4067349135875702,
 0.4029841423034668,
 0.413223475217819

### 모델 평가 추가

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import pandas as pd

In [15]:
# 준비된 수술 환자 데이터를 불러들입니다.
Data_set = pd.read_csv("./data/ThoraricSurgery.csv", header=None)

# 환자의 기록과 수술 결과를 X와 Y로 구분하여 저장합니다.
X = Data_set.iloc[:, :-1]
y = Data_set.iloc[:, -1]


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

In [16]:
# (1) 딥러닝 모델 설정 : 딥러닝 구조 결정
model = Sequential()
model.add(Dense(30, input_dim=17, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# (2) 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
# (3) 모델 실행(학습)
model.fit(X_train, Y_train, epochs=100, batch_size=10)

Epoch 1/100
33/33 [==============================] - 0s 749us/step - loss: 2.6360 - accuracy: 0.7508
Epoch 2/100
33/33 [==============================] - 0s 405us/step - loss: 0.9968 - accuracy: 0.7052
Epoch 3/100
33/33 [==============================] - 0s 374us/step - loss: 0.7612 - accuracy: 0.7781
Epoch 4/100
33/33 [==============================] - 0s 436us/step - loss: 0.6599 - accuracy: 0.8237
Epoch 5/100
33/33 [==============================] - 0s 436us/step - loss: 0.5947 - accuracy: 0.7994
Epoch 6/100
33/33 [==============================] - 0s 436us/step - loss: 0.5574 - accuracy: 0.7964
Epoch 7/100
33/33 [==============================] - 0s 405us/step - loss: 0.6660 - accuracy: 0.8055
Epoch 8/100
33/33 [==============================] - 0s 436us/step - loss: 0.4762 - accuracy: 0.8419
Epoch 9/100
33/33 [==============================] - 0s 405us/step - loss: 0.6070 - accuracy: 0.7964
Epoch 10/100
33/33 [==============================] - 0s 405us/step - loss: 0.4903 - accura

33/33 [==============================] - 0s 405us/step - loss: 0.4403 - accuracy: 0.8298
Epoch 82/100
33/33 [==============================] - 0s 436us/step - loss: 0.4041 - accuracy: 0.8511
Epoch 83/100
33/33 [==============================] - 0s 436us/step - loss: 0.4104 - accuracy: 0.8359
Epoch 84/100
33/33 [==============================] - 0s 436us/step - loss: 0.4619 - accuracy: 0.8116
Epoch 85/100
33/33 [==============================] - 0s 436us/step - loss: 0.4547 - accuracy: 0.8207
Epoch 86/100
33/33 [==============================] - 0s 436us/step - loss: 0.3906 - accuracy: 0.8298
Epoch 87/100
33/33 [==============================] - 0s 436us/step - loss: 0.4516 - accuracy: 0.8267
Epoch 88/100
33/33 [==============================] - 0s 436us/step - loss: 0.4152 - accuracy: 0.8176
Epoch 89/100
33/33 [==============================] - 0s 436us/step - loss: 0.4820 - accuracy: 0.8116
Epoch 90/100
33/33 [==============================] - 0s 405us/step - loss: 0.4585 - accuracy: 

#### 모델 평가
evaluate(x_test, y_test)[0/1]

In [18]:
# (4) 모델 평가
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

5/5 [==============================] - 0s 499us/step - loss: 0.6461 - accuracy: 0.8227

 Test Accuracy: 0.8227


In [19]:
## 모델 구성 확인
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 30)                540       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 31        
Total params: 571
Trainable params: 571
Non-trainable params: 0
_________________________________________________________________


#### 층을 추가해서 테스트

In [20]:
# (1) 딥러닝 모델 설정 : 딥러닝 구조 결정
model = Sequential()
model.add(Dense(30, input_dim=17, activation='relu'))
model.add(Dense(10, input_dim=17, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# (2) 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# (3) 모델 실행(학습)
model.fit(X_train, Y_train, epochs=100, batch_size=10)

# (4) 모델 평가
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test)[1]))

Epoch 1/100
33/33 [==============================] - 0s 530us/step - loss: 1.4860 - accuracy: 0.7264
Epoch 2/100
33/33 [==============================] - 0s 436us/step - loss: 0.6413 - accuracy: 0.8359
Epoch 3/100
33/33 [==============================] - 0s 405us/step - loss: 0.5437 - accuracy: 0.8298
Epoch 4/100
33/33 [==============================] - 0s 440us/step - loss: 0.4950 - accuracy: 0.8359
Epoch 5/100
33/33 [==============================] - 0s 436us/step - loss: 0.4815 - accuracy: 0.8359
Epoch 6/100
33/33 [==============================] - 0s 436us/step - loss: 0.4848 - accuracy: 0.8267
Epoch 7/100
33/33 [==============================] - 0s 436us/step - loss: 0.5417 - accuracy: 0.8328
Epoch 8/100
33/33 [==============================] - 0s 436us/step - loss: 0.4617 - accuracy: 0.8328
Epoch 9/100
33/33 [==============================] - 0s 467us/step - loss: 0.4757 - accuracy: 0.8359
Epoch 10/100
33/33 [==============================] - 0s 436us/step - loss: 0.4467 - accura

33/33 [==============================] - 0s 436us/step - loss: 0.4093 - accuracy: 0.8511
Epoch 82/100
33/33 [==============================] - 0s 436us/step - loss: 0.4059 - accuracy: 0.8419
Epoch 83/100
33/33 [==============================] - 0s 467us/step - loss: 0.4289 - accuracy: 0.8450
Epoch 84/100
33/33 [==============================] - 0s 436us/step - loss: 0.4155 - accuracy: 0.8359
Epoch 85/100
33/33 [==============================] - 0s 467us/step - loss: 0.4258 - accuracy: 0.8419
Epoch 86/100
33/33 [==============================] - 0s 436us/step - loss: 0.4049 - accuracy: 0.8389
Epoch 87/100
33/33 [==============================] - 0s 436us/step - loss: 0.4525 - accuracy: 0.8085
Epoch 88/100
33/33 [==============================] - 0s 436us/step - loss: 0.4535 - accuracy: 0.8267
Epoch 89/100
33/33 [==============================] - 0s 468us/step - loss: 0.4321 - accuracy: 0.8389
Epoch 90/100
33/33 [==============================] - 0s 467us/step - loss: 0.4203 - accuracy: 

5/5 [==============================] - 0s 748us/step - loss: 0.4887 - accuracy: 0.8582

 Test Accuracy: 0.8582


In [21]:
## 모델 구성 확인
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 30)                540       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                310       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 861
Trainable params: 861
Non-trainable params: 0
_________________________________________________________________
